In [3]:
import gym
import torch
import pandas as pd
from sb3_contrib import TRPO, RecurrentPPO, TQC
import numpy as np

In [11]:
class ClipReward(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.best_y = 6
        self.score = 0
        self.last_y = 6

    def reward(self, reward):
        ram = env.unwrapped.ale.getRAM()
        reward = 0
        '''
        reward -= (1/ram[14]) * 40



        if ram[103] > self.score:
            self.score = ram[103]
            reward += 1000

        if self.last_y > ram[14]:
            reward -= ram[14]'''
        if 90 <= ram[106] <= 100:
            reward -= 1
        if ram[14] > self.best_y:
            self.best_y = ram[14]
            reward += 3
        if ram[14] >= 172:
            self.best_y = 6
        return reward

In [ ]:
env = ClipReward(gym.make("ALE/Freeway-v5", obs_type="ram", difficulty=1, mode=0))
obs = env.reset()

model = TRPO("MlpPolicy", env, gamma=0.99, verbose=1, tensorboard_log="tensorlog")
#model = TRPO.load("trpo_5.zip", env=env)
model.learn(total_timesteps=1_000_000, log_interval=4)
model.save("trpo_1.zip")

model = TRPO.load("trpo_1.zip")

env = ClipReward(gym.make("ALE/Freeway-v5", render_mode="human", obs_type="ram", difficulty=1, mode=0))
obs = env.reset()
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    #env.render()
    #time.sleep(0.00001)
    print(reward)
    if done:
        obs = env.reset()

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to tensorlog\TRPO_9
----------------------------------------
| rollout/                  |          |
|    ep_len_mean            | 2.05e+03 |
|    ep_rew_mean            | 11.3     |
| time/                     |          |
|    fps                    | 366      |
|    iterations             | 4        |
|    time_elapsed           | 22       |
|    total_timesteps        | 8192     |
| train/                    |          |
|    explained_variance     | -0.84    |
|    is_line_search_success | 1        |
|    kl_divergence_loss     | 0.00278  |
|    learning_rate          | 0.001    |
|    n_updates              | 3        |
|    policy_objective       | 0.0138   |
|    value_loss             | 0.0979   |
----------------------------------------
----------------------------------------
| rollout/                  |          |
|    ep_len_mean            | 2.05e+03 |
|    ep_rew_mean

In [ ]:
model = TRPO.load("trpo_3.zip")

env = ClipReward(gym.make("ALE/Freeway-v5", render_mode="human", obs_type="ram", difficulty=1, mode=0))
obs = env.reset()
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    #env.render()
    #time.sleep(0.00001)
    print(reward)
    if done:
        obs = env.reset()

In [24]:
import gym
from gym.wrappers import Monitor

env = Monitor(ClipReward(gym.make("ALE/Freeway-v5", render_mode="human", obs_type="ram", difficulty=1, mode=0)), './video', force=True)
state = env.reset()

model = TRPO.load("trpo_1.zip", env=env)

done = False
for i in range(1000):
    action, _states = model.predict(state, deterministic=True)
    state, reward, done, info = env.step(action)
env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
